## 

In [1]:
import itertools
import numpy as np
from numpy.linalg import norm
from ase import Atoms
from ase.spacegroup import crystal
from ase.spacegroup import Spacegroup
from ase.data import atomic_numbers, atomic_names
from ase.io import read
from ase.visualize import view
import spglib
import nglview
from glob import glob

In [2]:
def ase_to_spgcell(ase_atoms=None, cell=None, inverse=False):
    if not inverse:
        assert ase_atoms is not None
        return (ase_atoms.get_cell(),
                ase_atoms.get_scaled_positions(),
                ase_atoms.get_atomic_numbers())
    else:
        assert cell is not None
        return Atoms(cell=cell[0],
                     scaled_positions=cell[1],
                     numbers=cell[2])

In [3]:
def a_equiv_b(a,b):
    """Function that identifies whether two crystals are equivalent"""

    # getting symmetry datasets for both crystals
    cryst_a = spglib.get_symmetry_dataset(ase_to_spgcell(ase_atoms=a), symprec=1e-5, angle_tolerance=-1.0, hall_number=0)
    cryst_b = spglib.get_symmetry_dataset(ase_to_spgcell(ase_atoms=b), symprec=1e-5, angle_tolerance=-1.0, hall_number=0)

    samecell = np.allclose(cryst_a['std_lattice'], cryst_b['std_lattice'], atol=1e-5)
    samenatoms = len(cryst_a['std_positions']) == len(cryst_b['std_positions'])
    samespg = cryst_a['number'] == cryst_b['number']
    
    def test_rotations_translations(cryst_a, cryst_b, repeat):
        cell = cryst_a['std_lattice']
        pristine = crystal('Mg', [(0, 0., 0.)], 
                           spacegroup=int(cryst_a['number']),
                           cellpar=[cell[0]/repeat[0], cell[1]/repeat[1], cell[2]/repeat[2]]).repeat(repeat)

        sym_set_p = spglib.get_symmetry_dataset(ase_to_spgcell(ase_atoms=pristine), symprec=1e-5,
                                               angle_tolerance=-1.0, hall_number=0)

        for _,trans in enumerate(zip(sym_set_p['rotations'], sym_set_p['translations'])):
            pnew=(np.matmul(trans[0],cryst_a['std_positions'].T).T + trans[1]) % 1.0
            fulln = np.concatenate([cryst_a['std_types'][:, None], pnew], axis=1)
            fullb = np.concatenate([cryst_b['std_types'][:, None], cryst_b['std_positions']], axis=1)
            sorted_n = np.array(sorted([ list(row) for row in list(fulln) ]))
            sorted_b = np.array(sorted([ list(row) for row in list(fullb) ]))
            if np.allclose(sorted_n, sorted_b, atol=1e-5):
                return True
        return False

    if samecell and samenatoms and samespg:
        cell = cryst_a['std_lattice']
        rng1 = range(1, int(norm(cell[0])/2.)) # here I assuem that the smallest possible crystal has a
        rng2 = range(1, int(norm(cell[1])/2.)) # unit cell of size 2
        rng3 = range(1, int(norm(cell[2])/2.))

        for repeat in itertools.product(rng1, rng2, rng3):
            if test_rotations_translations(cryst_a, cryst_b, repeat):
                return True

    return False

## Visualization of single structures

In [4]:
atoms = read('./quartz_alpha.xyz')
#atoms = read('./quartz_alpha_v2.xyz')
view(atoms,viewer='ngl')

In [5]:
v=_

In [6]:
# delete all old components
while hasattr(v.view, "component_0"):
    v.view.component_0.clear_representations()
    cid = v.view.component_0.id
    v.view.remove_component(cid)
v.view.add_component(nglview.ASEStructure(atoms), 
                     default_representation=False)
v.view.add_ball_and_stick(aspectRatio=2.0, opacity=1.0,component=0)
v.view.add_unitcell()
v.view.center()

In [7]:
cell = ase_to_spgcell(ase_atoms=atoms)
lattice, scaled_positions, numbers = spglib.find_primitive(cell, symprec=1e-5)
reduced = ase_to_spgcell(cell=(lattice, scaled_positions, numbers),inverse=True)

## Structure comparison

In [8]:
versions = [read(atoms) for atoms in glob('./quartz_alpha*')]

for outer in range(len(versions)-1):
    for inner in range(outer+1,len(versions)):
        print('Comparing: ',outer,inner)
        print('Equivalent: ',a_equiv_b(versions[outer],versions[inner]))

Comparing:  0 1
Equivalent:  False
Comparing:  0 2
Equivalent:  False
Comparing:  0 3
Equivalent:  False
Comparing:  0 4
Equivalent:  False
Comparing:  1 2
Equivalent:  False
Comparing:  1 3
Equivalent:  False
Comparing:  1 4
Equivalent:  False
Comparing:  2 3
Equivalent:  True
Comparing:  2 4
Equivalent:  False
Comparing:  3 4
Equivalent:  False
